# Getting the Data and Unzipping

In [1]:
import os
import cv2
from math import atan2, asin
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset, sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import albumentations as aug
from albumentations import (HorizontalFlip,VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise,RandomRotate90,Transpose,RandomBrightnessContrast, RandomCrop)
from albumentations.pytorch import ToTensor
import torch
from torchvision import transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.models as models
import time
import scipy.io
import random

seed = 23
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.set_default_tensor_type("torch.FloatTensor")

Getting the Path of train and test data

In [2]:
def TrainPathList(path):
  Img_Path = []
  for root, dirs, files in os.walk(path, topdown=False):
      for filename in files:
        if(filename[-3:] == 'jpg'):
          Img_Path.append(root+'/'+filename)
  return Img_Path        

In [3]:
def TestPathList(path):
  Img_Path = []
  for root, dirs, files in os.walk(path, topdown=False):
      for filename in files:
        if(filename[-3:] == 'jpg'):
          Img_Path.append(root+filename)
  return Img_Path        

In [4]:
TrainImagesPath = TrainPathList('300W_LP/')

In [5]:
TestImagesPath = TestPathList('AFLW2000/')

Function to get Yaw, Pitch and Roll using Rotation matrix 

In [6]:
# !git clone https://github.com/AryanRaj315/EfficientNet-PyTorch

In [7]:
def getAngles(path):
    mat = scipy.io.loadmat(path)
    angle_mat = mat['Pose_Para'][0]
    pitch = angle_mat[0]*180/math.pi
    yaw = angle_mat[1]*180/math.pi
    roll = angle_mat[2]*180/math.pi
    return np.array([yaw, pitch, roll]) 

# Class for Dataset

In [8]:
class WLP300Dataset(Dataset):
    def __init__(self, image_path):
        self.phase = 'train'
        self.path = image_path
        self.transforms = get_transforms(self.phase)

    def __getitem__(self, idx):
        Image = cv2.imread(self.path[idx])
        Angle = getAngles(self.path[idx][:-3]+'mat')
        augmented = self.transforms(image=Image)
        Image = augmented['image']
        return Image, Angle

    def __len__(self):
          return len(self.path)

In [9]:
class AFLWDataset(Dataset):
    def __init__(self, image_path, transforms= None):
        self.phase = 'test'
#         self.transform = transform
        self.path = image_path
        self.transforms = get_transforms(self.phase)

    def __getitem__(self, idx):
        Image = cv2.imread(self.path[idx]).astype('float32')
        Angle = getAngles(self.path[idx][:-3]+'mat')
        augmented = self.transforms(image=Image)
        Image = augmented['image'] 
        return Image, Angle

    def __len__(self):
          return len(self.path)

In [10]:
def get_transforms(phase):
    list_transforms = []
    if phase == "train":
        list_transforms.extend(
        [
         aug.OneOf([
             aug.RandomContrast(),
             aug.RandomGamma(),
             aug.RandomBrightness(),
             ], p=1),
         Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225], p=1)
        ]
    )
        #In case of test
    list_transforms.extend(
        [
            Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225], p=1),
            ToTensor(),
        ]
    )
    list_trfms = Compose(list_transforms)
    return list_trfms

In [11]:
def provider(phase, image_list, batch_size=8, num_workers=0):  
    if phase == 'test':
        image_dataset = AFLWDataset(image_list)
    else:    
        image_dataset = WLP300Dataset(image_list)
    dataloader = DataLoader(
        image_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=True,   
    )
    return dataloader

# Class for Training

In [12]:
class Trainer(object):
    '''This class takes care of training and validpathation of our model'''
    def __init__(self, model, Train_image_list,Test_image_list, bs, lr, epochs):
        self.batch_size = bs
        self.accumulation_steps = 1
        self.lr = lr
        self.num_epochs = epochs
        self.best_loss = float("inf")
        self.phases = ["train", "val"]
        self.device = torch.device("cuda:0")
        torch.set_default_tensor_type("torch.FloatTensor")
        self.net = model
        self.image_list = {'train': Train_image_list, 'val': Test_image_list}
        self.losses = {phase: [] for phase in self.phases}
        self.criterion = torch.nn.L1Loss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode="min", patience=3, verbose=True,factor = 0.5,min_lr = 1e-5)
#         self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max, eta_min=5e-5, last_epoch=-1)
        self.net = self.net.to(self.device)
        cudnn.benchmark = True
        self.dataloaders = {
            phase: provider(
                phase=phase,
                image_list = self.image_list[phase],
                batch_size=self.batch_size
            )
            for phase in self.phases
        }
        self.losses = {phase: [] for phase in self.phases}
        
    def forward(self, images, targets):
        images = images.to(self.device)
        targets = targets.type("torch.FloatTensor")
        targets = targets.to(self.device)
        yaw, pitch, roll = self.net(images)
        preds = torch.cat((yaw, pitch, roll), 1)
#         preds = torch.Tensor([yaw,pitch,roll])
        preds.to(self.device)
        loss = self.criterion(preds,targets)
#         loss_yaw = self.criterion(yaw, target[:,0])
#         loss_pitch = self.criterion(pitch, target[:,1])
#         loss_roll = self.criterion(roll, target[:,2])
#         loss_yaw = torch.mean(torch.stack([loss_pitch, loss_roll]))
        return loss

    def iterate(self, epoch, phase):
        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | phase: {phase} | ⏰: {start}")
        batch_size = self.batch_size
        self.net.train(phase == "train")
        dataloader = self.dataloaders[phase]
        running_loss = 0.0
        total_batches = len(dataloader)
        tk0 = tqdm(dataloader, total=total_batches)
        self.optimizer.zero_grad()
        for itr, batch in enumerate(tk0): # replace `dataloader` with `tk0` for tqdm
            images, targets = batch
            loss= self.forward(images, targets)
            loss = loss / self.accumulation_steps
            if phase == "train":
                loss.backward()
                if (itr + 1 ) % self.accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            running_loss += loss.item()
#             outputs = outputs.detach().cpu()
            tk0.set_postfix(loss=(running_loss / ((itr + 1))))
        epoch_loss = (running_loss * self.accumulation_steps) / total_batches
        torch.cuda.empty_cache()
        print(f'loss:{epoch_loss}')
        return epoch_loss

    def train_end(self):
        train_loss = self.losses["train"]
        val_loss = self.losses["val"]
        df_data=np.array([train_loss,val_loss]).T
        df = pd.DataFrame(df_data,columns = ['train_loss','val_loss'])
        df.to_csv("model-expand-att.csv")

    def start(self):
        for epoch in range(self.num_epochs):
            train_loss = self.iterate(epoch, "train")
            self.losses["train"].append(train_loss)
            state = {
                "epoch": epoch,
                "best_loss": self.best_loss,
                "state_dict": self.net.state_dict(),
                "optimizer": self.optimizer.state_dict(),
            }
            with torch.no_grad():
                val_loss = self.iterate(epoch, "val")
                self.losses["val"].append(val_loss)
                self.scheduler.step(val_loss)
            if val_loss < self.best_loss:
                print("******** New optimal found, saving state ********")
                state["best_loss"] = self.best_loss = val_loss
                torch.save(state, "./model-expand-att.pth")
            print()
            self.train_end()    


In [14]:
import sys
sys.path.insert(0, 'vision/')

In [13]:
!git clone 'https://github.com/AryanRaj315/vision.git'

Cloning into 'vision'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 6193 (delta 7), reused 7 (delta 3), pack-reused 6171
Receiving objects: 100% (6193/6193), 9.62 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (4128/4128), done.


In [15]:
from torchvision import models

In [16]:
model = models.resnet50()#.from_pretrained('efficientnet-b0')

In [16]:
# def matplotlib_imshow(img, one_channel=False):
#     if one_channel:
#         img = img.mean(dim=0)
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     if one_channel:
#         plt.imshow(npimg)
#     else:
#         plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [17]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter('runs/300W_LP')

In [18]:
# import torchvision

In [19]:
# trainloader = provider("train", TrainImagesPath)
# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # create grid of images
# img_grid = torchvision.utils.make_grid(images)

# # show images
# matplotlib_imshow(img_grid, one_channel=True)

In [20]:
# writer.add_image('300W_LP/', img_grid)

In [21]:
# writer.add_graph(model, images)
# writer.close()

In [22]:
# !tensorboard --logdir=runs 

In [23]:
# !wget 'http://storage.googleapis.com/public-models/efficientnet/efficientnet-b4-6ed6700e.pth'

In [ ]:
pretrained_dict = torch.load('efficientnet-b4-6ed6700e.pth')
IncompatibleKeys = model.load_state_dict(pretrained_dict,strict = False)

In [ ]:
UnloadedWeights = IncompatibleKeys[0][:-21]
AvailableWeights = IncompatibleKeys[1][8:]

In [ ]:
count = 0
for i in range(len(UnloadedWeights)):
    if AvailableWeights[count][-19:] == 'num_batches_tracked':
        count+=1
        model.state_dict().get(UnloadedWeights[i], None).data = pretrained_dict[AvailableWeights[count]]
    count += 1
    if(count == 109):
        count = 0

In [ ]:
num_ftrs = model._fc_p1.in_features
model._fc_p1 = nn.Linear(num_ftrs, 1)
model._fc_p2 = nn.Linear(num_ftrs, 1)
model._fc_p3 = nn.Linear(num_ftrs, 1)

In [ ]:
model_trainer = Trainer(model, TrainImagesPath, TestImagesPath, 8,1e-3,5)
model_trainer.start()

Starting epoch: 0 | phase: train | ⏰: 10:00:59


In [ ]:
# model = torch.load()